# Pre Notebook (Title In Progress)

<i>Authors: Rohan Pattnaik, Felix Martinez and the PyQSOFit Team</i>

---
Something about how to set up a catalog that is compatable with the fitting code but can have the users idk flush this part out more later with instructions and stuff

This notebook is divided into 3 sections:
- <b>Catalog Formatting: (Required) </b>
    - Where we allow the user to build a catalog of their data that is compatable with the fitting program. This section has an example with a subsection of the DEIMOS catalog.
    
    
- <b>Setting up a Configuration File: (Optional) </b>
    - Where we build a configuration file that allows the user to alter the initial conditions of the fitting program


- <b>Creating an Arguments file: (Optional) (WIP)</b>
    - Designed to build an arguments file that the fitting code will read to determine how it will fit.

## Catalog Formatting

### First Begining by Loading the Catalog
The catalog we will be using for this example is a subsample from the DEIMOS catalog. You can find it located in our `example_data` folder.

In [5]:
# Importing our packages
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# defining our homepath
hpath = os.getcwd()

# printing out the pathway
print('Home Path:',hpath)

Home Path: /Users/rohanpattnaik/My Drive/Rohan/Research/extragalactic/AutoQSOFit


In [46]:
# Loading in the catalog we will be using. The user can change the pathway to match their own catalog location
catalog = pd.read_csv(hpath+'/example_data/deimos_cat_subsample.csv', index_col=[0])

# displaying the first 5 rows of the DEIMOS catalog
catalog.head()

,DEIMOS_ID,ra,dec,sel,imag,kmag,zspec,Qf,Q,Remarks,fits1d,fits2d
0,L202441,150.227040,1.583884,160,20.00,19.00,0.345,4,2,"[OII]d,K,Hb,NaI,Ha,NIId,SIId","<a href=""/data/COSMOS/spectra/deimos/spec1d_fi...","<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
1,C1797605,150.808533,2.902927,128,19.85,23.07,0.365,14,2,"Hb(br),[OIII]d,Ha(br),NII,SiII","<a href=""/data/COSMOS/spectra/deimos/spec1d_fi...","<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
2,L329159,149.496458,1.789806,256,24.60,24.23,0.247,4,2,"[OIII]d,Ha","<a href=""/data/COSMOS/spectra/deimos/spec1d_fi...","<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
3,L385110,150.077576,1.875135,128,21.92,20.57,1.174,4,2,"MgII(abs),[OII],K,Hg","<a href=""/data/COSMOS/spectra/deimos/spec1d_fi...","<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
4,L421932,150.381638,1.932056,1,20.58,20.82,0.084,4,2,"Hb,[OIII]d,Ha,SII","<a href=""/data/COSMOS/spectra/deimos/spec1d_fi...","<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."


## Formatting the Catalog for Fitting:
This is the part where we make sure the catalog will be compatable with our program

What is most necessary for this section is the catalog have the following format:

|  | ID | ra (optional) | dec (optional) | z  | wavelength | flux | error |
| --- | -------- | ------------- | -------------- | - | - | - | -|
| 1 | id_number | ra | dec | redshift  | wavelength | flux | flux error |

### Units for Fitting:
Make sure that the units for fitting are as follows:
- <b>Wavelength</b>: angstrom
- <b>Flux</b>: 10^-17 [ergs/(s * cm^2 * angstrom)]
- <b>Flux Error</b>: 10^-17 [ergs/(s * cm^2 * angstrom)]

### Formatting our Wavelength, Flux, and Error from fits files
For our sample, we will be reading in fits files from the web.

We will be using the function `read_fits()` below to pull the data out of a fits file without keeping the file opened. Once the data is pulled out of the file, we can then assign it to our formatted catalog.

In [47]:
def read_fits(fits_dir):
    '''
    Reads in a fits file and outputs the data.
    ---
    Input: (fits file) 
        - The fits file that we want opened
    ---
    Output: (astropy.io.fits.fitsrec.FITS_rec) 
        - The data that is within the fits file
    '''

    # opening the fits file
    hdul  = fits.open(fits_dir)

    # copying the data of the fits file to a variable
    for hdu in hdul:
        try:
            image_data = hdu.data.copy()
        except:
            continue
            
    # closing the fits file
    hdul.close()

    return image_data

In [48]:
# Our sample dataset requires us to pull the fits1D data from the web. To do that we must ensure
# that each galaxy has a url that links to the fits file

# url pathway for spectra on DEIMOS website
deimos_path1D = 'https://irsa.ipac.caltech.edu/data/COSMOS/spectra/deimos/spec1d_fits/'

# renaming "fits_path_1D" column to include just the url pathway of the fits file
fits_path = [x[1] for x in catalog['fits1d'].str.split('"').to_list()]
fits_path = [deimos_path1D + x.split('/')[6] for x in fits_path]
catalog['fits1d'] = fits_path
catalog.head()

,DEIMOS_ID,ra,dec,sel,imag,kmag,zspec,Qf,Q,Remarks,fits1d,fits2d
0,L202441,150.227040,1.583884,160,20.00,19.00,0.345,4,2,"[OII]d,K,Hb,NaI,Ha,NIId,SIId",https://irsa.ipac.caltech.edu/data/COSMOS/spec...,"<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
1,C1797605,150.808533,2.902927,128,19.85,23.07,0.365,14,2,"Hb(br),[OIII]d,Ha(br),NII,SiII",https://irsa.ipac.caltech.edu/data/COSMOS/spec...,"<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
2,L329159,149.496458,1.789806,256,24.60,24.23,0.247,4,2,"[OIII]d,Ha",https://irsa.ipac.caltech.edu/data/COSMOS/spec...,"<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
3,L385110,150.077576,1.875135,128,21.92,20.57,1.174,4,2,"MgII(abs),[OII],K,Hg",https://irsa.ipac.caltech.edu/data/COSMOS/spec...,"<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."
4,L421932,150.381638,1.932056,1,20.58,20.82,0.084,4,2,"Hb,[OIII]d,Ha,SII",https://irsa.ipac.caltech.edu/data/COSMOS/spec...,"<a href=""/data/COSMOS/spectra/deimos/spec2d/sp..."


In [49]:
# Looping through our catalog to pull data from the fits files

wavelength, flux, error = [], [], []
for i in range(len(catalog)):
    
    # pulling the data from the fits file
    output = read_fits(catalog.fits1d[i])
    
    ########## These variables will change based on your fits file ##########
    
    # pulling out wavelength
    wavelength.append(output['LAMBDA'][0])     # units of Angstrom
    
    # pulling out flux
    flux.append(output['FLUX'][0])             # units of 10^-17 [ergs/(s * cm^2 * Angstrom)]
    
    # calculating error
    ivar = output['IVAR'][0]                   # Error represented in inverse variance
    error.append(ivar)                         # Convert to error from ivar (same units as flux)

### Combining all our data into the formatted catalog
Now that we have all the data we need, we will format our catalog to match that of the fitting process. The format necessary is listed above.

In [50]:
## Building a DataFrame that matches the column names we need
formatted_cat = pd.DataFrame()

# To avoid errors, do not change the formatted_cat column names
formatted_cat['ID'] = catalog['DEIMOS_ID']
formatted_cat['ra'] = catalog['ra']
formatted_cat['dec'] = catalog['dec']
formatted_cat['z'] = catalog['zspec']
formatted_cat['wavelength'] = wavelength
formatted_cat['flux'] = flux
formatted_cat['error'] = error

formatted_cat.head()

,ID,ra,dec,z,wavelength,flux,error
0,L202441,150.227040,1.583884,0.345,"[4474.72314453125, 4475.35400390625, 4475.9853...","[0.0, 0.0, 0.0, 10.639404296875, -1.7802734375...","[0.0, 0.0, 0.0, 0.004625608678907156, 0.002511..."
1,C1797605,150.808533,2.902927,0.365,"[5913.13525390625, 5913.603515625, 5914.071289...","[0.0, 0.0, 3.984375476837158, 15.2260551452636...","[0.0, 0.0, 168.15231323242188, 16.535520553588..."
2,L329159,149.496458,1.789806,0.247,"[6036.107421875, 6036.56005859375, 6037.012695...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,L385110,150.077576,1.875135,1.174,"[5946.80908203125, 5947.26171875, 5947.7138671...","[0.0, 0.0, 4.203361988067627, 5.27965164184570...","[0.0, 0.0, 0.010846253484487534, 0.27086707949..."
4,L421932,150.381638,1.932056,0.084,"[5086.701171875, 5087.3095703125, 5087.9179687...","[0.0, 0.0, 0.0, 179.52516174316406, 95.7243576...","[0.0, 0.0, 0.0, 6.7869941631215625e-06, 0.0005..."


<b>We now have a catalog that is formatted for fitting. Run the cell below to save it in the `catalogs` folder. </b>

In [87]:
### Only run if saving catalog
formatted_cat.to_pickle('catalogs/input_catalog.pkl')

## Setting up a Configuration File
Here we build a configuration file that the fitting code will call upon.

In [1]:
# Importing our packages
# Standard packages
import glob, os, sys, timeit
import pandas as pd
from astropy.io import fits
from astropy.table import Table

# Scipy packages
import scipy.interpolate as sc
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

# Fitting packages
from pyqsofit.PyQSOFit import QSOFit
from astropy.convolution import convolve, Gaussian1DKernel

# System packages
from multiprocessing import Pool,cpu_count
import warnings
warnings.filterwarnings("ignore")

### Creating a Prior for Emission Lines (Optional)

Here we built a fits file that specifies the priors/initial conditions and boundaries for the line fitting parameters in QSOFit.

This part is presented by the original PyQSOFit example notebook which can be found [here](https://github.com/legolason/PyQSOFit/blob/master/example/example.ipynb).

In [2]:
# create a header
hdr0 = fits.Header()
hdr0['Author'] = 'Hengxiao Guo'
primary_hdu = fits.PrimaryHDU(header=hdr0)

In [6]:
"""
In this table, we specify the priors / initial conditions and boundaries for the line fitting parameters.
"""

line_priors = np.rec.array([
    (6564.61, 'Ha', 6400, 6800, 'Ha_br', 2, 0.0, 0.0, 1e10, 5e-3, 0.004, 0.05, 0.015, 0, 0, 0, 0.05, 1),
    (6564.61, 'Ha', 6400, 6800, 'Ha_na', 1, 0.0, 0.0, 1e10, 1e-3, 5e-4, 0.00169, 0.01, 1, 1, 0, 0.002, 1),
    (6549.85, 'Ha', 6400, 6800, 'NII6549', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 5e-3, 1, 1, 1, 0.001, 1),
    (6585.28, 'Ha', 6400, 6800, 'NII6585', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 5e-3, 1, 1, 1, 0.003, 1),
    (6718.29, 'Ha', 6400, 6800, 'SII6718', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 5e-3, 1, 1, 2, 0.001, 1),
    (6732.67, 'Ha', 6400, 6800, 'SII6732', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 5e-3, 1, 1, 2, 0.001, 1),

    (4862.68, 'Hb', 4640, 5100, 'Hb_br', 2, 0.0, 0.0, 1e10, 5e-3, 0.004, 0.05, 0.01, 0, 0, 0, 0.01, 1),
    (4862.68, 'Hb', 4640, 5100, 'Hb_na', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 0.01, 1, 1, 0, 0.002, 1),
    (4960.30, 'Hb', 4640, 5100, 'OIII4959c', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 0.01, 1, 1, 0, 0.002, 1),
    (5008.24, 'Hb', 4640, 5100, 'OIII5007c', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 0.01, 1, 1, 0, 0.004, 1),
    (4960.30, 'Hb', 4640, 5100, 'OIII4959w',   1, 0.0, 0.0, 1e10, 3e-3, 2.3e-4, 0.004,  0.01,  2, 2, 0, 0.001, 1),
    (5008.24, 'Hb', 4640, 5100, 'OIII5007w',   1, 0.0, 0.0, 1e10, 3e-3, 2.3e-4, 0.004,  0.01,  2, 2, 0, 0.002, 1),
    (4687.02, 'Hb', 4640, 5100, 'HeII4687_br', 1, 0.0, 0.0, 1e10, 5e-3, 0.004,  0.05,   0.005, 0, 0, 0, 0.001, 1),
    (4687.02, 'Hb', 4640, 5100, 'HeII4687_na', 1, 0.0, 0.0, 1e10, 1e-3, 2.3e-4, 0.00169, 0.005, 1, 1, 0, 0.001, 1),

    (3934.78, 'CaII', 3900, 3960, 'CaII3934' , 2, 0.0, 0.0, 1e10, 1e-3, 3.333e-4, 0.00169, 0.01, 99, 0, 0, -0.001, 1),

    (3728.48, 'OII', 3650, 3800, 'OII3728', 1, 0.0, 0.0, 1e10, 1e-3, 3.333e-4, 0.00169, 0.01, 1, 1, 0, 0.001, 1),

    (3426.84, 'NeV', 3380, 3480, 'NeV3426',    1, 0.0, 0.0, 1e10, 1e-3, 3.333e-4, 0.00169, 0.01, 0, 0, 0, 0.001, 1),
    (3426.84, 'NeV', 3380, 3480, 'NeV3426_br', 1, 0.0, 0.0, 1e10, 5e-3, 0.0025,   0.02,   0.01, 0, 0, 0, 0.001, 1),

    (2798.75, 'MgII', 2700, 2900, 'MgII_br', 2, 0.0, 0.0, 1e10, 5e-3, 0.004, 0.05, 0.015, 0, 0, 0, 0.05, 1),
    (2798.75, 'MgII', 2700, 2900, 'MgII_na', 1, 0.0, 0.0, 1e10, 1e-3, 5e-4, 0.00169, 0.01, 1, 1, 0, 0.002, 1),

    (1908.73, 'CIII', 1700, 1970, 'CIII_br', 2, 0.0, 0.0, 1e10, 5e-3, 0.004, 0.05, 0.015, 99, 0, 0, 0.01, 1),
    (1908.73, 'CIII', 1700, 1970, 'CIII_na',   1, 0.0, 0.0, 1e10, 1e-3, 5e-4,  0.00169, 0.01,  1, 1, 0, 0.002, 1),
    (1892.03, 'CIII', 1700, 1970, 'SiIII1892', 1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.003, 1, 1, 0, 0.005, 1),
    (1857.40, 'CIII', 1700, 1970, 'AlIII1857', 1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.003, 1, 1, 0, 0.005, 1),
    (1816.98, 'CIII', 1700, 1970, 'SiII1816',  1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.01,  1, 1, 0, 0.0002, 1),
    (1786.7,  'CIII', 1700, 1970, 'FeII1787',  1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.01,  1, 1, 0, 0.0002, 1),
    (1750.26, 'CIII', 1700, 1970, 'NIII1750',  1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.01,  1, 1, 0, 0.001, 1),
    (1718.55, 'CIII', 1700, 1900, 'NIV1718',   1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015,  0.01,  1, 1, 0, 0.001, 1),

    (1549.06, 'CIV', 1500, 1700, 'CIV_br', 2, 0.0, 0.0, 1e10, 5e-3, 0.004, 0.05, 0.015, 0, 0, 0, 0.05, 1),
    (1549.06, 'CIV', 1500, 1700, 'CIV_na', 1, 0.0, 0.0, 1e10, 1e-3, 5e-4, 0.00169, 0.01, 1, 1, 0, 0.002, 1),
    (1640.42, 'CIV', 1500, 1700, 'HeII1640',    1, 0.0, 0.0, 1e10, 1e-3, 5e-4,   0.00169, 0.008, 1, 1, 0, 0.002, 1),
    (1663.48, 'CIV', 1500, 1700, 'OIII1663',    1, 0.0, 0.0, 1e10, 1e-3, 5e-4,   0.00169, 0.008, 1, 1, 0, 0.002, 1),
    (1640.42, 'CIV', 1500, 1700, 'HeII1640_br', 1, 0.0, 0.0, 1e10, 5e-3, 0.0025, 0.02,   0.008, 1, 1, 0, 0.002, 1),
    (1663.48, 'CIV', 1500, 1700, 'OIII1663_br', 1, 0.0, 0.0, 1e10, 5e-3, 0.0025, 0.02,   0.008, 1, 1, 0, 0.002, 1),

    (1402.06, 'SiIV', 1290, 1450, 'SiIV_OIV1', 1, 0.0, 0.0, 1e10, 5e-3, 0.002, 0.05,  0.015, 1, 1, 0, 0.05, 1),
    (1396.76, 'SiIV', 1290, 1450, 'SiIV_OIV2', 1, 0.0, 0.0, 1e10, 5e-3, 0.002, 0.05,  0.015, 1, 1, 0, 0.05, 1),
    (1335.30, 'SiIV', 1290, 1450, 'CII1335',   1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015, 0.01,  1, 1, 0, 0.001, 1),
    (1304.35, 'SiIV', 1290, 1450, 'OI1304',    1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.015, 0.01,  1, 1, 0, 0.001, 1),

    (1215.67, 'Lya', 1150, 1290, 'Lya_br', 3, 0.0, 0.0, 1e10, 5e-3, 0.002, 0.05, 0.02, 0, 0, 0, 0.05, 1),
    (1240.14, 'Lya', 1150, 1290, 'NV1240', 1, 0.0, 0.0, 1e10, 2e-3, 0.001, 0.01, 0.005, 0, 0, 0, 0.002, 1),
    (1215.67, 'Lya', 1150, 1290, 'Lya_na', 1, 0.0, 0.0, 1e10, 1e-3, 5e-4, 0.00169, 0.01, 0, 0, 0, 0.002, 1),
    ],

    formats='float32,    a20,  float32, float32,      a20,  int32, float32, float32, float32, float32, float32, float32, float32,   int32,  int32,  int32, float32, int32',
    names=' lambda, compname,   minwav,  maxwav, linename, ngauss,  inisca,  minsca,  maxsca,  inisig,  minsig,  maxsig,    voff,  vindex, windex, findex,  fvalue,  vary')

In [7]:
# Header
hdr1 = fits.Header()
hdr1['lambda'] = 'Vacuum Wavelength in Ang'
hdr1['minwav'] = 'Lower complex fitting wavelength range'
hdr1['maxwav'] = 'Upper complex fitting wavelength range'
hdr1['ngauss'] = 'Number of Gaussians for the line'

# Can be set to negative for absorption lines if you want
hdr1['inisca'] = 'Initial guess of line scale [flux]'
hdr1['minsca'] = 'Lower range of line scale [flux]'
hdr1['maxsca'] = 'Upper range of line scale [flux]'

hdr1['inisig'] = 'Initial guess of linesigma [lnlambda]'
hdr1['minsig'] = 'Lower range of line sigma [lnlambda]'
hdr1['maxsig'] = 'Upper range of line sigma [lnlambda]'

hdr1['voff  '] = 'Limits on velocity offset from the central wavelength [lnlambda]'
hdr1['vindex'] = 'Entries w/ same NONZERO vindex constrained to have same velocity'
hdr1['windex'] = 'Entries w/ same NONZERO windex constrained to have same width'
hdr1['findex'] = 'Entries w/ same NONZERO findex have constrained flux ratios'
hdr1['fvalue'] = 'Relative scale factor for entries w/ same findex'

hdr1['vary'] = 'Whether or not to vary the parameter (set to 0 to fix the line parameter to initial values)'

# Save line info
hdu1 = fits.BinTableHDU(data=line_priors, header=hdr1, name='line_priors')

In [8]:
"""
In this table, we specify the windows and priors / initial conditions and boundaries for the continuum fitting parameters.
"""

conti_windows = np.rec.array([
    (1150., 1170.), 
    (1275., 1290.),
    (1350., 1360.),
    (1445., 1465.),
    (1690., 1705.),
    (1770., 1810.),
    (1970., 2400.),
    (2480., 2675.),
    (2925., 3400.),
    (3775., 3832.),
    (4000., 4050.),
    (4200., 4230.),
    (4435., 4640.),
    (5100., 5535.),
    (6005., 6035.),
    (6110., 6250.),
    (6800., 7000.),
    (7160., 7180.),
    (7500., 7800.),
    (8050., 8150.), # Continuum fitting windows (to avoid emission line, etc.)  [AA]
    ], 
    formats = 'float32,  float32',
    names =    'min,     max')

hdu2 = fits.BinTableHDU(data=conti_windows, name='conti_windows')

conti_priors = np.rec.array([
    ('Fe_uv_norm',  0.0,   0.0,   1e10,  1), # Normalization of the MgII Fe template [flux]
    ('Fe_uv_FWHM',  3000,  1200,  18000, 1), # FWHM of the MgII Fe template [AA]
    ('Fe_uv_shift', 0.0,   -0.01, 0.01,  1), # Wavelength shift of the MgII Fe template [lnlambda]
    ('Fe_op_norm',  0.0,   0.0,   1e10,  1), # Normalization of the Hbeta/Halpha Fe template [flux]
    ('Fe_op_FWHM',  3000,  1200,  18000, 1), # FWHM of the Hbeta/Halpha Fe template [AA]
    ('Fe_op_shift', 0.0,   -0.01, 0.01,  1), # Wavelength shift of the Hbeta/Halpha Fe template [lnlambda]
    ('PL_norm',     1.0,   0.0,   1e10,  1), # Normalization of the power-law (PL) continuum f_lambda = (lambda/3000)^-alpha
    ('PL_slope',    -1.5,  -5.0,  3.0,   1), # Slope of the power-law (PL) continuum
    ('Blamer_norm', 0.0,   0.0,   1e10,  1), # Normalization of the Balmer continuum at < 3646 AA [flux] (Dietrich et al. 2002)
    ('Balmer_Te',   15000, 10000, 50000, 1), # Te of the Balmer continuum at < 3646 AA [K?]
    ('Balmer_Tau',  0.5,   0.1,   2.0,   1), # Tau of the Balmer continuum at < 3646 AA
    ('conti_a_0',   0.0,   None,  None,  1), # 1st coefficient of the polynomial continuum
    ('conti_a_1',   0.0,   None,  None,  1), # 2nd coefficient of the polynomial continuum
    ('conti_a_2',   0.0,   None,  None,  1), # 3rd coefficient of the polynomial continuum
    # Note: The min/max bounds on the conti_a_0 coefficients are ignored by the code,
    # so they can be determined automatically for numerical stability.
    ],

    formats = 'a20,  float32, float32, float32, int32',
    names = 'parname, initial,   min,     max,     vary')

hdr3 = fits.Header()
hdr3['ini'] = 'Initial guess of line scale [flux]'
hdr3['min'] = 'FWHM of the MgII Fe template'
hdr3['max'] = 'Wavelength shift of the MgII Fe template'

hdr3['vary'] = 'Whether or not to vary the parameter (set to 0 to fix the continuum parameter to initial values)'


hdu3 = fits.BinTableHDU(data=conti_priors, header=hdr3, name='conti_priors')

In [9]:
"""
In this table, we allow user to customized some key parameters in our result measurements.
"""

measure_info = Table(
    [
        [[1350, 1450, 3000, 4200, 5100]],
        [[
            # [2240, 2650], 
            [4435, 4685],
        ]]
    ],
    names=([
        'cont_loc',
        'Fe_flux_range'
    ]),
    dtype=([
        'float32',
        'float32'
    ])
)
hdr4 = fits.Header()
hdr4['cont_loc'] = 'The wavelength of continuum luminosity in results'
hdr4['Fe_flux_range'] = 'Fe emission wavelength range calculated in results'

hdu4 = fits.BinTableHDU(data=measure_info, header=hdr4, name='measure_info')

hdu_list = fits.HDUList([primary_hdu, hdu1, hdu2, hdu3, hdu4])

We now can save this fits files in the `pyqsofit` folder as `qsopar.fits`.

In [10]:
# saving the line profile
hdu_list.writeto('pyqsofit/qsopar.fits', overwrite=True)

### Creating an Arguments File for PyQSOFit (Optional) (WIP)

Here we build an arguments file that the fitting code will read to determine how it will fit. All the options are listed below including what they do. The fitting code will pull from this file to determine how it will fit the galaxies you feed into it. 

The defaults are listed: (we will go into more detail here later. So far the defaults are listed in the `autoqsofit.py` file underneath the `q.fit()` function)

In [2]:
tbl = pd.read_pickle('catalogs/input_catalog.pkl')
tbl

,ID,ra,dec,z,wavelength,flux,error
0,L202441,150.227040,1.583884,0.3450,"[4474.72314453125, 4475.35400390625, 4475.9853...","[0.0, 0.0, 0.0, 10.639404296875, -1.7802734375...","[0.0, 0.0, 0.0, 0.004625608678907156, 0.002511..."
1,C1797605,150.808533,2.902927,0.3650,"[5913.13525390625, 5913.603515625, 5914.071289...","[0.0, 0.0, 3.984375476837158, 15.2260551452636...","[0.0, 0.0, 168.15231323242188, 16.535520553588..."
2,L329159,149.496458,1.789806,0.2470,"[6036.107421875, 6036.56005859375, 6037.012695...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,L385110,150.077576,1.875135,1.1740,"[5946.80908203125, 5947.26171875, 5947.7138671...","[0.0, 0.0, 4.203361988067627, 5.27965164184570...","[0.0, 0.0, 0.010846253484487534, 0.27086707949..."
4,L421932,150.381638,1.932056,0.0840,"[5086.701171875, 5087.3095703125, 5087.9179687...","[0.0, 0.0, 0.0, 179.52516174316406, 95.7243576...","[0.0, 0.0, 0.0, 6.7869941631215625e-06, 0.0005..."
5,L452180,150.105194,1.981136,0.3720,"[6332.54541015625, 6332.962890625, 6333.380371...","[2.6926028728485107, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2015.13671875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,L739170,150.100979,2.419435,4.6335,"[6287.71044921875, 6288.16259765625, 6288.6152...","[0.0, 0.0, -5.45703125, 18.48193359375, -1.831...","[0.0, 0.0, 0.04006835073232651, 0.011695243418..."
7,L636831,149.965454,2.265508,0.7512,"[5986.37109375, 5986.826171875, 5987.28125, 59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,L864422,149.709820,2.610581,0.8280,"[5191.75244140625, 5192.36328125, 5192.9741210...","[0.0, 0.0, 0.0, 16.140625, 33.474609375, 24.99...","[0.0, 0.0, 0.0, 0.0062587670981884, 0.00099977..."
9,L301504,150.137730,1.743644,0.9810,"[4990.771484375, 4991.38232421875, 4991.993164...","[0.0, 0.0, 0.0, 0.515625, 29.7275390625, -89.1...","[0.0, 0.0, 0.0, 0.007434762082993984, 0.001168..."


In [3]:
for index,row in tbl.iterrows():
    pass

In [4]:
row['ID']

'L301504'

In [5]:
final_lines = pd.read_pickle('results/0_L202441_0.345/0_L202441_0.345_table.pkl')
final_lines

,Linename,ngauss,type,compcenter,scale_centerwave_sigma,all_scale_centerwave_sigma,FWHM,FWHM_err,Sigma,Sigma_err,...,Peak,Peak_err,flux,flux_err,snr,all_FWHM,all_Sigma,all_EW,all_Peak,all_flux
0,CaII3934,2,narrow,3934.780029,"[0.0, 8.277609825134277, 0.0010000000474974513...","[[0.0, 8.277609825134277, 0.001000000047497451...",0.000000,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,nan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Ha_br,2,broad,6564.609863,"[80.96974701832238, 8.774450587026964, 0.00649...","[[81.00657422627222, 8.77445050467971, 0.00649...",4515.201265,1.723843e+02,1891.784696,7.226665e+01,...,6466.889969,1.054258e-01,17005.861777,4.636964e+02,36.674563,"[4515.302274159469, 4515.198390243768, 4515.20...","[1891.8237318668205, 1891.7802017546492, 1891....","[108.79720237203401, 108.71901057029916, 128.6...","[6466.8899686843215, 6466.8899686843215, 6466....","[8502.700543657891, 8496.595168578166, 10051.7..."
2,Ha_na,1,narrow,6564.609863,"[2087.24791173065, 8.789679130570244, 0.000500...","[[2087.2479256084375, 8.789679130562575, 0.000...",353.244225,2.847315e-07,147.665070,1.400091e-07,...,6566.124969,1.818989e-12,17121.183729,1.856044e+02,92.245566,"[353.2442255923702, 353.24422557957746, 353.24...","[147.66506973883196, 147.66506984406112, 147.6...","[163.55768704906225, 163.55767882209653, 163.5...","[6566.124968641393, 6566.124968641393, 6566.12...","[17121.183858065986, 17121.182996869025, 17121..."
3,Hb_br,2,broad,4862.680176,"[0.0, 8.48934555053711, 0.004999999888241291, ...","[[0.0, 8.48934555053711, 0.004999999888241291,...",0.000000,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,nan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Hb_na,1,narrow,4862.680176,"[1.1102230246251565e-06, 8.48934555053711, 0.0...","[[1.1102230246251565e-06, 8.48934555053711, 0....",709.631195,1.136868e-13,295.652268,1.136868e-13,...,4862.099165,0.000000e+00,0.000013,0.000000e+00,nan,"[709.631194839978, 709.631194839978, 709.63119...","[295.65226774813823, 295.65226774813823, 295.6...","[1.3172015941649225e-07, 1.3172015941649225e-0...","[4862.099164687531, 4862.099164687531, 4862.09...","[1.3494194611500775e-05, 1.3494194611500775e-0..."
5,HeII4687_br,1,broad,4687.020020,"[0.0, 8.45255184173584, 0.004999999888241291]","[[0.0, 8.45255184173584, 0.004999999888241291]...",0.000000,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,nan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,HeII4687_na,1,narrow,4687.020020,"[0.0, 8.45255184173584, 0.0010000000474974513]","[[0.0, 8.45255184173584, 0.0010000000474974513...",0.000000,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,nan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,NII6549,1,narrow,6549.850098,"[207.26197758058618, 8.787427505509209, 0.0005...","[[230.4886292936459, 8.78742750550154, 0.00050...",353.243975,2.847363e-07,147.664965,1.513857e-07,...,6551.357149,1.818989e-12,1696.295546,5.438276e+01,31.191789,"[353.243975011808, 353.2439749990281, 353.2439...","[147.66496518058327, 147.66496533913